In [ ]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import pypsa
import pyomo.environ as pyo
import pyomo.version
import shapely
import tqdm
print('pandas', pd.__version__)
print('geopandas', gpd.__version__)
print('xarray', xr.__version__)
print('pypsa', pypsa.__version__)
print('pyomo', getattr(pyo, '__version__', pyomo.version.version))
print('shapely', shapely.__version__)
print('tqdm', tqdm.__version__)

In [ ]:
from importlib import reload
import sys
from pathlib import Path

def find_repo_root(start: Path) -> Path:
    start = start.resolve()
    for p in [start] + list(start.parents):
        if (p / "model").is_dir():
            return p
    raise RuntimeError("Could not locate repo root containing 'model/'")

NOTEBOOK_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
repo_root = find_repo_root(NOTEBOOK_DIR)

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from model import main as plant_main

plant_main = reload(plant_main)

plant_dir = repo_root / "basic_ammonia_plant"
net = plant_main.generate_network(24, plant_dir, aggregation_count=1, time_step=1.0)
print(f"Network components: {len(net.links)} links, {len(net.generators)} generators, {len(net.stores)} stores")

In [ ]:
from importlib import reload
from pathlib import Path
import logging
import numbers
import os
import sys

def find_repo_root(start: Path) -> Path:
    start = start.resolve()
    for p in [start] + list(start.parents):
        if (p / "model").is_dir():
            return p
    raise RuntimeError("Could not locate repo root containing 'model/'")

NOTEBOOK_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
repo_root = find_repo_root(NOTEBOOK_DIR)

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from model import auxiliary as aux
from model import main as plant_main

aux = reload(aux)
plant_main = reload(plant_main)

logging.getLogger("pypsa").setLevel(logging.WARNING)
logging.getLogger("pypsa.optimization").setLevel(logging.ERROR)
logging.getLogger("pypsa.optimization.constraints").setLevel(logging.ERROR)
logging.getLogger("linopy").setLevel(logging.WARNING)
logging.getLogger("linopy.io").setLevel(logging.WARNING)
logging.getLogger("gurobipy").setLevel(logging.WARNING)

weather_file = repo_root / "data" / "single_site_weather_data.csv"
weather_data = aux.get_weather_data(file_name=str(weather_file), aggregation_count=1)
snap_len = len(weather_data)
solver = "gurobi"
results = plant_main.main(file_name=str(weather_file), multi_site=True, aggregation_count=1, time_step=1.0)

print("Single-location run summary:")
print(f"  Weather file: {weather_file.relative_to(repo_root)}")
print(f"  Snapshots: {snap_len} (aggregation_count=1)")
print(f"  Solver: {solver}")
lcoa = results.get("lcoa_usd_per_t")
if lcoa is not None:
    print(f"  LCOA: {lcoa:.2f} USD/t")

print("Key outputs (MW for capacities, MWh for energy stores):")
for name, value in results.items():
    if name in {"lcoa_usd_per_t", "annual_ammonia_demand_mwh", "annual_ammonia_production_t", "total_cost_usd_per_year"} or name.startswith("cost_share_") or name.startswith("lcoa_component_") or name.startswith("interest_rate_") or name.endswith("_share_pct"):
        continue
    if isinstance(value, numbers.Real):
        formatted_value = f"{value:,.2f}"
    else:
        formatted_value = value
    unit = "MWh" if name in {"ammonia", "compressed_hydrogen_store", "battery", "accumulated_penalty"} else "MW"
    if name == "hydrogen_storage_capacity":
        unit = "t"
    print(f"  {name}: {formatted_value} {unit}")

### Single-case diagnostic run
Re-run the single-location optimisation while keeping the solved `pypsa.Network` object in memory so we can inspect time-series behaviour before scaling to multi-area batches.

In [ ]:
import matplotlib.pyplot as plt

# Build a fresh network with the full weather series length
single_case_net = plant_main.generate_network(
    len(weather_data),
    plant_dir,
    aggregation_count=1,
    time_step=1.0,
)

renewables = set(single_case_net.generators.index)
for column in weather_data.columns:
    if column in renewables:
        single_case_net.generators_t.p_max_pu[column] = weather_data[column].values

status, condition = single_case_net.optimize(
    solver_name=solver,
    extra_functionality=aux.linopy_constraints,
)
if status != "ok":
    raise RuntimeError(f"Diagnostic optimisation failed: {status} ({condition})")

diagnostic_results = aux.get_results_dict_for_multi_site(
    single_case_net,
    aggregation_count=1,
    time_step=1.0,
)
print(f"Diagnostic LCOA: {diagnostic_results['lcoa_usd_per_t']:.2f} USD/t")

In [ ]:
timeline_df = pd.DataFrame(index=single_case_net.snapshots)

for gen_name in ("wind", "solar", "solar_tracking"):
    if gen_name in single_case_net.generators.index:
        timeline_df[gen_name] = single_case_net.generators_t.p[gen_name]

if "electrolysis" in single_case_net.links_t.p0.columns:
    timeline_df["electrolysis_power"] = -single_case_net.links_t.p0["electrolysis"]
if "ammonia_synthesis" in single_case_net.links_t.p0.columns:
    timeline_df["ammonia_synthesis_power"] = -single_case_net.links_t.p0["ammonia_synthesis"]

palette = {
    "solar": "#f28e2b",
    "wind": "#9ad0f5",
    "solar_tracking": "#76b7b2",
    "electrolysis_power": "#4e79a7",
    "ammonia_synthesis_power": "#0f5c2f",
}

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
gen_cols = [c for c in ("wind", "solar", "solar_tracking") if c in timeline_df]
if gen_cols:
    ordered_gen_cols = sorted(gen_cols, key=lambda c: timeline_df[c].abs().mean(), reverse=True)
    colors = [palette.get(col, None) for col in ordered_gen_cols]
    timeline_df[ordered_gen_cols].plot(ax=axes[0], color=colors)
    axes[0].set_ylabel("Power (MW)")
    axes[0].set_title("Renewable production profiles")
    axes[0].legend(loc="upper right")
else:
    axes[0].set_visible(False)

consumption_cols = [c for c in ("electrolysis_power", "ammonia_synthesis_power") if c in timeline_df]
if consumption_cols:
    colors = [palette.get(col, None) for col in consumption_cols]
    timeline_df[consumption_cols].plot(ax=axes[1], linestyle="-", linewidth=1.0, color=colors)
    axes[1].set_ylabel("Power consumption (MW)")
    axes[1].set_title("Process power demand")
    axes[1].legend(loc="upper right")
axes[-1].set_xlabel("Snapshot")
plt.tight_layout()

#### Ammonia ramp-rate check
Confirm the solved dispatch respects the ±0.4 per-unit constraints encoded in `basic_ammonia_plant/links.csv`.

In [ ]:
ammonia_series = -single_case_net.links_t.p0["ammonia_synthesis"]
installed_capacity = single_case_net.links.loc["ammonia_synthesis", "p_nom_opt"]
ramp_limit = single_case_net.links.loc["ammonia_synthesis", "ramp_limit_up"]
max_allowed_delta = ramp_limit * installed_capacity
observed_delta = ammonia_series.diff().abs().max()
print(f"Installed ammonia capacity: {installed_capacity:,.2f} MW")
print(f"Ramp limit (per snapshot): {ramp_limit:.2f} pu -> {max_allowed_delta:,.2f} MW")
print(f"Observed max change: {observed_delta:,.2f} MW")
print("Constraint respected?", observed_delta <= max_allowed_delta + 1e-6)

In [ ]:
penalty_energy = single_case_net.stores_t.e["accumulated_penalty"]
print("Max accumulated_penalty energy (MWh):", penalty_energy.max())
print("Min accumulated_penalty energy (MWh):", penalty_energy.min())

### NetCDF ingestion sanity checks
Recreate the weather frame from the NetCDF stacks for a known site and confirm those values are exactly what the model receives.

In [ ]:
from model import location_tools as lt
from model import run_global

weather_dir = repo_root / "data"
nc_dataset = lt.all_locations(str(weather_dir))
sample_lat, sample_lon = (-23.0, 133.0)
nc_aggregation_count = 1

nc_weather_frame = run_global._build_weather_frame(
    nc_dataset,
    lat=sample_lat,
    lon=sample_lon,
    aggregation_count=nc_aggregation_count,
)

profile_cols = [col for col in ("wind", "solar", "solar_tracking") if col in nc_weather_frame.columns]
if not profile_cols:
    raise ValueError("No renewable columns found in NetCDF frame.")

print(f"Loaded NetCDF profiles for ({sample_lat}, {sample_lon}) with {len(nc_weather_frame)} snapshots.")
display(nc_weather_frame[profile_cols].describe().T[["mean", "min", "max"]])
nc_weather_frame.head()

In [ ]:
from types import MethodType


def build_network_with_nc_profiles(weather_frame, aggregation_count=1):
    network = plant_main.generate_network(
        len(weather_frame),
        "basic_ammonia_plant",
        aggregation_count=aggregation_count,
        time_step=1.0,
    )

    plant_main.apply_weather_profiles(network, weather_frame)

    def _stub_optimize(self, *args, **kwargs):
        return ("ok", "skipped for profile check")

    network.optimize = MethodType(_stub_optimize, network)
    return network


nc_profile_net = build_network_with_nc_profiles(nc_weather_frame, aggregation_count=nc_aggregation_count)
comparison = []
for column in profile_cols:
    series_diff = (nc_profile_net.generators_t.p_max_pu[column] - nc_weather_frame[column]).abs()
    comparison.append({"component": column, "max_abs_diff": float(series_diff.max())})

comparison_df = pd.DataFrame(comparison).set_index("component")
display(comparison_df)
assert comparison_df["max_abs_diff"].max() < 1e-9
print("Weather frame from NetCDF matches the values pushed into the network.")